# Laboratorio: Auth Encryption
Gustavo Méndez, Michele Benvenuto, Randy Venegas, Luis Urbina, Roberto Figueroa

**GCM**


In [18]:
!pip install pycryptodome

Cifrado de mensaje con GCM

In [19]:
#Implementación Galois-Counter Mode (GCM)
#libraries
import json
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

#message to encrypt
data = b"Aqui ten mando Q100000.00 para la inversion"

#header (aditional data)
header = b"Sender:192.168.1.1Reciever:192.168.1.2SequenceNumber:15542342"

#Generate a 16-bytes (128 bits) long key for AES
key = get_random_bytes(16)

#New AES-GCM mode cipher (cipher and authentication)
cipher = AES.new(key, AES.MODE_GCM)

#additional data to just authenticate
cipher.update(header)

#encrypt and authenticate
ciphertext, tag = cipher.encrypt_and_digest(data)

#setting list for json keys
json_k = [ 'nonce', 'header', 'ciphertext', 'tag' ]

#parsing values to utf-8 
json_v = [ b64encode(x).decode('utf-8') for x in [cipher.nonce, header, ciphertext, tag] ]

#pretty printing
result = json.dumps(dict(zip(json_k, json_v)), indent=2)
print("key is:", bytes(key))
print("The result is: \n", result)

key is: b'\xfe\xe0\x9bq\x18\x0e\xb8\x85\x0eA\xb4iG\xbc\xbe\xfb'
The result is: 
 {
  "nonce": "nWFErQGjmkHx03hh2CGZnw==",
  "header": "U2VuZGVyOjE5Mi4xNjguMS4xUmVjaWV2ZXI6MTkyLjE2OC4xLjJTZXF1ZW5jZU51bWJlcjoxNTU0MjM0Mg==",
  "ciphertext": "K9p/WcBfn6Nitn+Ep65n+xiFEYDl+tlTyn/zsaIixisR1im3dcKYexG0Kg==",
  "tag": "5nipGwrkXCWHgEIAq85yzg=="
}


Ahora procederemos a la descifrado del mensaje

In [20]:
import json
from base64 import b64decode
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
# We assume that the key was securely shared beforehand
key = b'\xc2\xdd\x88[\x1b\xa7\xea\xbf\x01\xeb[p\xfeF\xe1\x10'

json_input =  {
  "nonce": "VPmd0mWkvGC/HuogAaEoJw==",
  "header": "U2VuZGVyOjE5Mi4xNjguMS4xUmVjaWV2ZXI6MTkyLjE2OC4xLjJTZXF1ZW5jZU51bWJlcjoxNTU0MjM0Mg==",
  "ciphertext": "cKqeww6bn8Pz6L8Hh13cB9y2WQSsNwsUCXixkPfgefO9YIkqzHVmAWkbow==",
  "tag": "rhl+lv4vWY5CuG49/q+T6g=="
}

 #Tries to decrypt and verify the integrity of the message
try:
    json_input = json.dumps(json_input)
    b64 = json.loads(json_input)
    #setting list for json keys
    json_k = [ 'nonce', 'header', 'ciphertext', 'tag' ]
    #getting the values for each key
    jv = {k:b64decode(b64[k]) for k in json_k}
    #New AES-GCM mode cipher object(cipher and authentication)
    cipher = AES.new(key, AES.MODE_GCM, nonce=jv['nonce'])
    #additional data that had been authenticated
    cipher.update(jv['header'])
    #decrypt the message and verify 
    plaintext = cipher.decrypt_and_verify(jv['ciphertext'], jv['tag'])
    print("The plain text is: ",plaintext.decode("utf-8"))
 #If the message can not be decrypted or it has been modified it raise an exception
except (ValueError, KeyError) as e:
    print("Incorrect decryption, it could be that the message has been modified", e)

The plain text is:  Aqui ten mando Q100000.00 para la inversion


**CCM**

In [9]:
!pip install pycryptodome

In [27]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# Encryption, message from Alice

hdr = b'This is for Bob'
plaintext = b'This is a secret secret message, Bob.'
key = b'Sixteen byte key'
nonce = get_random_bytes(11)
cipher = AES.new(key, AES.MODE_CCM, nonce)
cipher.update(hdr)
msg = nonce, hdr, cipher.encrypt(plaintext), cipher.digest()

# Decryption, Bob (receiver) gets the ``msg``

nonce, hdr, ciphertext, mac = msg
key = b'Sixteen byte key'
cipher = AES.new(key, AES.MODE_CCM, nonce)
cipher.update(hdr)
plaintext = cipher.decrypt(ciphertext)
try:
    cipher.verify(mac)
    print("The message is authentic: hdr=%s, pt=%s", (hdr, plaintext))
except ValueError:
    print("Key incorrect or message corrupted")


The message is authentic: hdr=%s, pt=%s (b'This is for Bob', b'This is a secret secret message, Bob.')


**EAX**

In [17]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# setting key and encryption
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(b'This is my secret secret string')

file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
file_out.close()
print("Key :",key)


file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)
print("Decrypted: ", data)

Key : b'\xf9\xc9\x17\x87\x88;\x0bg|\x06T\xdf\r\xf0\x15\xd0'
Decrypted:  b'This is my secret secret string'


**PBKDF2**

In [ ]:
# We need to install backports.pbkdf2 on Google Colab Notebook
!pip install backports.pbkdf2

In [ ]:
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac, compare_digest

# Initialize salt and password
salt = binascii.unhexlify('aaef2d3f4d77ac66e9c5a6c3d8f921d1')
passwd = "my-password".encode("utf8")
# Working with SHA-256 for calculating HMAC, 25000 iterations and finally get a 
# 32-bytes derivated key 
key = pbkdf2_hmac("sha256", passwd, salt, 50000, 32)
print("Final Key:", binascii.hexlify(key))

Final Key: b'5dbb161307331364cb5418109b5cbef5b8a110dbe13be947c67ff2096b5d468e'


**Argon2**

In [ ]:
from argon2 import PasswordHasher

# Instancia de la clase PasswordHasher
ph = PasswordHasher(time_cost=2, memory_cost=102400, parallelism=8, hash_len=16, salt_len=16, encoding='utf-8')

# Obtención del Hash
hash = ph.hash("s3kr3tp4ssw0rd")

# Imprimir el hash
print("El Hash es: " + hash)

# Verificar si el hash hace match con la contraseña definida previamente
print(ph.verify(hash, "s3kr3tp4ssw0rd"))

El Hash es: $argon2id$v=19$m=102400,t=2,p=8$O3AeiTNBBSXzsCIBbctMNQ$BREz9g0DcTQnDsQ/xe7Z8g
True
